In [ ]:
# Imports
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split
import numpy as np
from google.colab import files


In [ ]:
# Upload dataset. Dataset is stored in file fens_dataset_shuf.
uploaded = files.upload()

Saving fens_dataset_shuf to fens_dataset_shuf


In [ ]:
# List of input and output data from dataset.
input_data = []
output_data = []

# Load dataset to memory.
# Every line in dataset contains chess position written in Fen notation and expected predicton.
# Example line: 
# 4r3/p1p2p1k/2b2b1p/5P2/6B1/PP6/2P2K2/6R1 w - - 3 35 : 0 0 1
# Input and output are seperated with colon.
# Output has three values. Only one value can be 1 and others 0.
# First value means white is wining, second means it's a draw and third black is winning.
with open('fens_dataset_shuf', 'r') as file:
    for line in file:
        line = line[:-1].strip().split(' : ')
        fen = line[0]
        if fen not in input_data:
            input_data.append(fen)
            scores = line[1].split(' ')
            output_data.append([float(i) for i in scores])

file.close              

<function TextIOWrapper.close>

In [ ]:
# Parse fen notation to array.
# Method extracts 70 features from fen position.
# First 64 represents chess board, last 6 values represent following:
# 1. White or black move (1/0)
# 2. Can white castle kingside (1/0)
# 3. Can white castle queenside (1/0)
# 4. Can black castle kingside (1/0)
# 5. Can black castle queenside (1/0)
# 6. En passant allowed (1/0)
# Returns:
#  - 8x8 numpy array
#  - 1x6 numpy array
def parse_fen(fen):
    
    input_data = []
    index = 0
    split = fen.split(' ')
    
    # First part, loop through fen to fill chess board with pieces.
    for i in range(len(fen)):
        # If character is space, break the loop, part with pieces is finished.
        if fen[i] == ' ':
            index += 1
            break

        # If character is slash, go to next row.    
        if fen[i] == '/':
            index += 1
            continue

        # If character is in chess_pieces, append its value.    
        if fen[i] in chess_pieces:
            input_data.append(chess_pieces[fen[i]])
            index += 1

        # Character is a number that represents next N empty squares.    
        else:
            for i in range(int(fen[i])):
                input_data.append(chess_pieces['.'])
            index += 1
    
    # Second part, for position state.
    # White's/black's move.
    if fen[index] == 'w':
        input_data.append(1)
    else:
        input_data.append(0)
        
    # Castling
    # skip space
    index += 1
    if fen[index] == '-':
        input_data.append(0)
        input_data.append(0)
        input_data.append(0)
        input_data.append(0)
        index += 4
    else:
        castle = split[2]
        if 'K' in castle:
            input_data.append(1)
        else:
            input_data.append(0)
        if 'Q' in castle:
            input_data.append(1)
        else:
            input_data.append(0)
        if 'k' in castle:
            input_data.append(1)
        else:
            input_data.append(0)
        if 'q' in castle:
            input_data.append(1)
        else:
            input_data.append(0)
     
    # En passant
    if split[3] != '-':
        input_data.append(1)
    else:
        input_data.append(0)
        
    return input_data, np.asarray(input_data[:64]).reshape((8,8, 12)), np.asarray(input_data[64:]) 

# Values for chess pieces.
chess_pieces = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

In [ ]:
# Parse fens from dataset
inputs = []
input_boards = []
input_states = []
for i in input_data:
    input, input_board, input_state = parse_fen(i)
    input_boards.append(input_board)
    input_states.append(input_state)
    inputs.append(input)

input_boards = np.asarray(input_boards)
input_states = np.asarray(input_states)

In [ ]:
# Build model
# Model input is 1D array of length 70.
# First part of the model is CNN, where first 64 input values are reshaped to 8x8 2D array.
# Second part is normal ANN, where last six input values are appended to output of CNN.

board_input = Input(shape=(8, 8, 12), name="board")
state_input = Input(shape=(6,), name='state')
x = layers.Conv2D(filters=512, kernel_size=8, padding="same", activation="relu", 
                  input_shape=(8,8,12))(board_input)
x = layers.Conv2D(filters=512, kernel_size=4, padding="same", activation="relu")(x)
x = layers.Conv2D(filters=512, kernel_size=2, padding="same", activation="relu")(x)
x = layers.Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=1024, kernel_size=1, padding="same", activation="relu")(x) 
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
merge = concatenate([x, state_input])
x = layers.Dense(1024, activation="relu")(merge)
x = layers.Dense(512, activation="relu")(x)
board_output = layers.Dense(3, activation='softmax')(x)


model = Model(inputs=[board_input, state_input], outputs=board_output, name="prediction_conv")


In [ ]:
# Compile model.
model.compile(
  optimizer=optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)


In [ ]:
# train model
model.fit(
  [input_boards, input_states],
  np.asarray(output_data),
  epochs=7,
  batch_size=256,
  validation_split=0.2,
)

Epoch 1/7
1152/1152 [==============================] - 173s 138ms/step - loss: 0.6336 - accuracy: 0.7263 - val_loss: 0.5836 - val_accuracy: 0.7567
Epoch 2/7
1152/1152 [==============================] - 167s 145ms/step - loss: 0.4631 - accuracy: 0.8110 - val_loss: 0.4468 - val_accuracy: 0.8180
Epoch 3/7
1152/1152 [==============================] - 169s 147ms/step - loss: 0.3766 - accuracy: 0.8498 - val_loss: 0.4387 - val_accuracy: 0.8249
Epoch 4/7
1152/1152 [==============================] - 169s 147ms/step - loss: 0.3113 - accuracy: 0.8776 - val_loss: 0.4336 - val_accuracy: 0.8341
Epoch 5/7
1152/1152 [==============================] - 169s 147ms/step - loss: 0.2660 - accuracy: 0.8962 - val_loss: 0.4374 - val_accuracy: 0.8392
Epoch 6/7
1152/1152 [==============================] - 169s 147ms/step - loss: 0.2252 - accuracy: 0.9125 - val_loss: 0.4635 - val_accuracy: 0.8327
Epoch 7/7
1152/1152 [==============================] - 169s 147ms/step - loss: 0.1961 - accuracy: 0.9239 - val_loss: 0

In [ ]:
model.save_weights('My_weights_h5',save_format='h5')

In [ ]:
# predictions
position, board, state = parse_fen('8/p1r2kpp/4pq2/8/2p1Pn2/1P3PR1/P1PQ1KPP/3R4 w - - 0 36')

boards = []
states = []

boards.append(board)
states.append(state)

boards

predictions = model.predict([np.asarray(boards), np.asarray(states)])
print(predictions)
print("Class " + str(np.argmax(predictions, axis=1)[0]))

[[8.3735540e-05 2.0525846e-04 9.9971098e-01]]
Class 2


In [ ]:
# save model
model.save_weights('model.h7')